# TP : Algorithmes itératifs pour les processus de décision Markovien

## 1 - Programmation dynamique à horizon ﬁni

### Question 1 
Vous devez tout d’abord écrire une structure de données d’arbre pondéré comme celui de la
ﬁgure ci-dessus.

In [1]:
# Classe représentant notre graphe
class Graph:
    
    def __init__(self):
        self.edges = {} # Seule les arrêtes sont enregistrées
        self.value_functions = {} # Stockage des fonctions de valeur
    
    # Une fonction qui facilite le stockage des arrêtes dans un dictionnaire
    def to_string(self,k1, s1, k2, s2):
        return str(k1)+','+str(s1)+' -- '+str(k2)+','+str(s2)
    
    # Stockage du poids de chaque arrête
    def add_edge(self,k1, s1, k2, s2, e):
        self.edges[self.to_string(k1, s1, k2, s2)] = e
        
    # Affichage du graph
    def print_graph(self):
        print('Niceau,Etat -- Niveau,Etat : poid')
        for k in self.edges.keys():
            print(k,':',self.edges[k])

In [2]:

g = Graph() # Initialisation du graphe

# Ajout des différents arrêtes du graph
# (Niveau, Etat, Niveau, Etat, poid)
g.add_edge(0, 1, 1, 1, 1)
g.add_edge(0, 1, 1, 2, 5)
g.add_edge(0, 1, 1, 3,-3)
g.add_edge(1, 1, 2, 1,-2)
g.add_edge(1, 1, 2, 2, 5)
g.add_edge(1, 2, 2, 1, 2)
g.add_edge(1, 2, 2, 2, 7)
g.add_edge(1, 2, 2, 3, 4)
g.add_edge(1, 3, 2, 2, 7)
g.add_edge(1, 3, 2, 3, 3)
g.add_edge(2, 1, 3, 1, 3)
g.add_edge(2, 1, 3, 2, 5)
g.add_edge(2, 2, 3, 1,-2)
g.add_edge(2, 2, 3, 2, 1)
g.add_edge(2, 2, 3, 3, 2)
g.add_edge(2, 3, 3, 2, 0)
g.add_edge(2, 3, 3, 3, 4)
g.add_edge(3, 1, 4, 1, 0)
g.add_edge(3, 2, 4, 1, 0)
g.add_edge(3, 3, 4, 1, 0)

# Affichage du graphe de la manière suivante: "Niveau,Etat -- Niveau,Etat : poid"
g.print_graph()

Niceau,Etat -- Niveau,Etat : poid
0,1 -- 1,1 : 1
0,1 -- 1,2 : 5
0,1 -- 1,3 : -3
1,1 -- 2,1 : -2
1,1 -- 2,2 : 5
1,2 -- 2,1 : 2
1,2 -- 2,2 : 7
1,2 -- 2,3 : 4
1,3 -- 2,2 : 7
1,3 -- 2,3 : 3
2,1 -- 3,1 : 3
2,1 -- 3,2 : 5
2,2 -- 3,1 : -2
2,2 -- 3,2 : 1
2,2 -- 3,3 : 2
2,3 -- 3,2 : 0
2,3 -- 3,3 : 4
3,1 -- 4,1 : 0
3,2 -- 4,1 : 0
3,3 -- 4,1 : 0


### Question 2 
Un premier programme déterminera par la méthode de récursivité inverse la fonction de valeur
dans chaque état du graphe pour chaque niveau de l’arbre.

In [3]:
# Fonction permettant de retourner l'ensemble des actions qu'on peut prendre à partir de l'étant courant s et du niveau k
def prev_edges(k,s):
    edges = list(g.edges.keys()) # Récupération de toutes les arrêtes 
    items = []
    for e in edges:
        if '- '+str(k)+','+str(s) in e: # Vérification si l'état courant est la destination dans l'arrête
            items.append(e)
    return items

# Fonction permettant de déterminer par la méthode de récursivité inverse la fonction de valeur dans chaque état du graphe 
# pour chaque niveau de l’arbre.
def V_star(k,s):
    if k == 0:
        val= 0
    else:
        val = max([g.edges[e] + V_star(int(e[0]),int(e[2])) for e in prev_edges(k,s)]) # Appel récursif de V_star
    g.value_functions['V*(k='+str(k)+',s='+str(s)+')'] = val
    return val

In [4]:
# Calcul des différente fonction de valeur à partir de k = 4, s= 1
V_star(4,1)
for k in g.value_functions.keys():
    print(k,'=',g.value_functions[k])

V*(k=0,s=1) = 0
V*(k=1,s=1) = 1
V*(k=1,s=2) = 5
V*(k=2,s=1) = 7
V*(k=1,s=3) = -3
V*(k=2,s=2) = 12
V*(k=3,s=1) = 10
V*(k=2,s=3) = 9
V*(k=3,s=2) = 13
V*(k=3,s=3) = 14
V*(k=4,s=1) = 14


### Question 3
Un second programme calculera la politique optimale et donc le chemine le plus long depuis la racine (k = 0) jusqu’aux feuilles (k = T).

In [5]:
# Fonction permettant de déterminer la politique optimale

def compute_politique_optimale(k,s):
    politique = [] # Stockage de la politique optimale
    temp1 = g.value_functions['V*(k='+str(k)+',s='+str(s)+')'] # Récupération de la fonction de valeur de la destination k = 4, s= 1
    politique.insert(0,'(k=4,s=1)')
    node =  ''
    
    # Tant que nous ne somme pas arrivé à l'état d'origine k = 0, s= 1, 
    # continuer par déterminer la prochaine action qui possède le maximum comme fonction de valeur
    while k != 0:
        temp2 = prev_edges(k,s)
        temp3 = [g.value_functions['V*(k='+e[0]+',s='+e[2]+')'] for e in temp2]
        val = temp3[0]
        node = temp2[0]
        for index, e in enumerate(temp3[1:], start = 1):
            if e > val:
                node = temp2[index]
                val = e
        k = int(node[0])
        s = int(node[2])
        politique.insert(0,'(k='+str(k)+',s='+str(s)+')')
    return politique

In [6]:
# Affichage du résultat jusqu'à k = 4, s = 1
result = compute_politique_optimale(4,1)
for k in result[:-1]:
    print(k)
    print('    | \n    | \n    v')
print(result[len(result)-1])
print('Le chemin le plus long est : V*(k=4,s=1) =',g.value_functions['V*(k='+str(4)+',s='+str(1)+')'])

(k=0,s=1)
    | 
    | 
    v
(k=1,s=2)
    | 
    | 
    v
(k=2,s=2)
    | 
    | 
    v
(k=3,s=3)
    | 
    | 
    v
(k=4,s=1)
Le chemin le plus long est : V*(k=4,s=1) = 14


## 2 - Programmation dynamique à horizon inﬁni

### Question 4 
Un premier programme correspond à l’algorithme de programmation dynamique par itération
de la valeur.

In [1]:
# Libraries import
import numpy as np
import math

In [2]:
# Data initialisation 
n = 3
m = 4
gamma = 0.9
start = np.array([2,0])
directions = {'LEFT':np.array([0,-1]),'UP':np.array([-1,0]),'RIGHT':np.array([0,1]),'DOWN':np.array([1,0])}
actions = np.array(['LEFT','RIGHT','UP','DOWN'])

In [3]:
# Board initialisation
V_board = np.zeros((n,m))
V_board[0,m-1] = 1
V_board[1,m-1] = -1
V_board[1,1] = math.nan
V_board_prev = V_board.copy()
forbidden_positions = [(1,1),(0,3),(1,3)]
print(V_board)

[[ 0.  0.  0.  1.]
 [ 0. nan  0. -1.]
 [ 0.  0.  0.  0.]]


In [4]:
# Function to check if a state is a wall
def isWall(position):
    if position[0] < 0 or position[0] >= n or position[1] < 0 or position[1] >= m or math.isnan(V_board[position[0],position[1]]):
        return True
    else:
        return False

In [5]:
# Function that compute the sum  ∑s′∈S p(s′|s, a)Vn(s′)]
def sum_P_V(position, action):
    temp = 0
    next_position = position + directions[action]
    if isWall(next_position):
        temp = temp + V_board_prev[position[0],position[1]]*0.8
    else:
        temp = temp + V_board_prev[next_position[0],next_position[1]]*0.8
        
    perpendicular_diraction = []
    if action == 'LEFT' or action == 'RIGHT':
        perpendicular_diraction.append('UP')
        perpendicular_diraction.append('DOWN')
    else:
        perpendicular_diraction.append('LEFT')
        perpendicular_diraction.append('RIGHT')
            
    
    for a in perpendicular_diraction:
        next_position = position + directions[a]
        if isWall(next_position):
            temp = temp + V_board_prev[position[0],position[1]]*0.1
        else:
            temp = temp + V_board_prev[next_position[0],next_position[1]]*0.1
    return temp

In [6]:
# Value iteration function
# We assume that the reward is 0 for all other states
def value_iteration(position):
    return  gamma*max([sum_P_V(position,a) for a in actions])

In [7]:
# #Test of Value Iteration

# nbIteration = 100
# for i in range(nbIteration):
#     for k in range(n):
#         for l in range(m):
#             if (k,l) not in forbidden_positions:
#                 V_board[k,l] = round(value_iteration([k,l]),3)
#     V_board_prev = V_board.copy()

# np.set_printoptions(suppress=True)

# print('Number of iteration :', nbIteration)
# print(V_board)


Number of iteration : 100
[[ 0.644  0.744  0.848  1.   ]
 [ 0.565    nan  0.572 -1.   ]
 [ 0.489  0.429  0.475  0.277]]


### Question 5
Un second programme correspond à la détermination de la politique optimale par itération de
la politique.

In [8]:
# Policy iteration function
def policy_iteration(position):
    sum_P_V_list = [sum_P_V(position,a) for a in actions]
    sum_P_V_max = max(sum_P_V_list)
    policy = actions[np.argmax(sum_P_V_list)]
    return  gamma*sum_P_V_max, policy

In [9]:
# Initialisation of policy board
def init_policy_board():
    policy_board = []
    for k in range(n):
        policy_board.append([])
        for l in range(m): 
            policy_board[k].append("     ")
    policy_board = np.array(policy_board)      
    policy_board[0,3] = 'UP'
    policy_board[1,3] = 'LEFT'
    policy_board_prev = policy_board.copy()
    
    return policy_board, policy_board_prev

In [10]:
Test of Policy iteration

policy_board, policy_board_prev = init_policy_board()
step = 1
while True:
    for k in range(n):
        for l in range(m):
            if (k,l) not in forbidden_positions:
                result = policy_iteration([k,l])
                V_board[k,l] = round(result[0],3)
                policy_board[k,l]= result[1]
    V_board_prev = V_board.copy()
    step += 1
    if step != 1 and np.array_equal(policy_board,policy_board_prev):
        break
    policy_board_prev = policy_board.copy()
print('Number of iteration : ',step)
print(policy_board)

Number of iteration :  7
[['RIGHT' 'RIGHT' 'RIGHT' 'UP']
 ['UP' '     ' 'UP' 'LEFT']
 ['UP' 'RIGHT' 'UP' 'LEFT']]
